In [ ]:
import time
import os

def test_write_speed(file_path="test_file.dat", file_size=1 * 1024 * 1024 * 1024):
    """Tests the write speed of creating a file with specified size."""
    data = b"0" * (1024 * 1024)
    total_written = 0
    start_time = time.time()

    with open(file_path, "wb") as f:
        while total_written < file_size:
            f.write(data)
            total_written += len(data)

    end_time = time.time()
    duration = end_time - start_time
    print(duration)
    speed = file_size / duration / (1024 * 1024)

    print(f"Write speed: {speed:.2f} MB/s")
    os.remove(file_path)

if __name__ == "__main__":
    test_write_speed()

5.626342296600342
Write speed: 182.00 MB/s


In [ ]:
!pip install liburing

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.4/117.4 kB 4.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for liburing: filename=liburing-2024.5.3-cp311-cp311-linux_x86_64.whl size=2351534 sha256=5c69d78d59ca5a649a17f0024fbf46dd75fb34e5761f51105bf29d82abd063b4
  Stored in directory: /root/.cache/pip/wheels/0e/d4/40/2239396498ad6f682f9e8ea9f4901d334fb693933a5b415fb2
Successfully built liburing


In [ ]:
# import time
# import os
# import liburing
# import mmap

# def test_write_speed(file_path="test_file.dat", file_size=1 * 1024 * 1024 * 1024):
#     """Tests the write speed of creating a file with specified size using io_uring."""
#     data_size = 1024 * 1024  # 1MB of data
#     queue_depth = 32  # Number of asynchronous requests

#     # Setup io_uring
#     ring = liburing.io_uring()
#     liburing.io_uring_queue_init(queue_depth, ring, 0)

#     # Open file
#     fd = os.open(file_path, os.O_WRONLY | os.O_CREAT | os.O_TRUNC, 0o644)

#     start_time = time.time()
#     with mmap.mmap(-1, data_size) as buf:
#         buf.write(b"0" * data_size)
#         buf.seek(0)

#         total_written = 0
#         while total_written < file_size:
#             # Get a submission queue entry (SQE)
#             sqe = liburing.io_uring_get_sqe(ring)

#             # Prepare the write operation
#             liburing.io_uring_prep_write(sqe, fd, buf, data_size, total_written)
#             total_written += data_size
#             liburing.io_uring_submit(ring)

#             # Ensure completion of write requests
#             cqe_ptr = liburing.io_uring_cqe()
#             liburing.io_uring_wait_cqe(ring, cqe_ptr)
#             liburing.io_uring_cqe_seen(ring, cqe_ptr)

#     # Cleanup
#     liburing.io_uring_queue_exit(ring)
#     os.close(fd)

#     end_time = time.time()
#     duration = end_time - start_time
#     speed = file_size / duration / (1024 * 1024)  # Convert to MB/s
#     print(duration)
#     # print(f"Write speed: {speed:.2f} MB/s")

# if __name__ == "__main__":
#     test_write_speed()


5.993032455444336


In [ ]:
import time
import os
import liburing
import mmap


def io_uring_write(file_path, data):
    """Efficiently writes data to a file using io_uring with batching."""
    queue_depth = 32  # Max number of requests in flight
    chunk_size = 4096  # Write in 4KB chunks for efficiency
    data_size = len(data)

    # Setup io_uring
    ring = liburing.io_uring()
    liburing.io_uring_queue_init(queue_depth, ring, 0)

    # Open file
    fd = os.open(file_path, os.O_WRONLY | os.O_CREAT | os.O_TRUNC, 0o644)

    # Use bytearray instead of mmap for efficient buffer management
    total_written = 0
    while total_written < data_size:
        submitted = 0

        # Queue multiple write requests up to queue_depth
        while submitted < queue_depth and total_written < data_size:
            sqe = liburing.io_uring_get_sqe(ring)
            if not sqe:
                break  # No more SQEs available

            # Determine chunk size
            write_size = min(chunk_size, data_size - total_written)

            # Prepare the write operation
            buf = data[total_written: total_written + write_size]
            liburing.io_uring_prep_write(sqe, fd, buf, write_size, total_written)
            total_written += write_size
            submitted += 1

        # Submit all queued requests at once
        if submitted > 0:
            liburing.io_uring_submit(ring)

        # Process completion queue
        completed = 0
        while completed < submitted:
            cqe_ptr = liburing.io_uring_cqe()
            if liburing.io_uring_wait_cqe(ring, cqe_ptr) == 0:
                liburing.io_uring_cqe_seen(ring, cqe_ptr)
                completed += 1

    # Cleanup
    liburing.io_uring_queue_exit(ring)
    os.close(fd)


def test_write_speed(file_path="test_file.dat", file_size=1 * 1024 * 1024 * 1024):
    """Tests the write speed of creating a file with specified size using io_uring."""
    data = b"0" * file_size
    start_time = time.time()
    io_uring_write(file_path, data)
    end_time = time.time()

    duration = end_time - start_time
    speed = file_size / duration / (1024 * 1024)  # Convert to MB/s

    print(f"Write speed: {speed:.2f} MB/s")
    os.remove(file_path)  # Clean up after test
    print(duration)

if __name__ == "__main__":
    test_write_speed()


Write speed: 175.83 MB/s
5.823745012283325


In [ ]:
import time
import os
import liburing
import mmap


import os
import liburing

def io_uring_write_all(file_path, data):
    """Writes all data at once to the specified file using io_uring."""
    queue_depth = 1  # Single request since we're writing all data at once
    data_size = len(data)

    # Setup io_uring
    ring = liburing.io_uring()
    liburing.io_uring_queue_init(queue_depth, ring, 0)

    # Open file
    fd = os.open(file_path, os.O_WRONLY | os.O_CREAT | os.O_TRUNC, 0o644)

    # Get a submission queue entry (SQE)
    sqe = liburing.io_uring_get_sqe(ring)

    if not sqe:
        raise RuntimeError("Failed to get SQE")

    # Prepare a single write operation for the entire data buffer
    liburing.io_uring_prep_write(sqe, fd, data, data_size, 0)

    # Submit the request
    liburing.io_uring_submit(ring)

    # Wait for the completion event
    cqe_ptr = liburing.io_uring_cqe()
    liburing.io_uring_wait_cqe(ring, cqe_ptr)
    liburing.io_uring_cqe_seen(ring, cqe_ptr)

    # Cleanup
    liburing.io_uring_queue_exit(ring)
    os.close(fd)



def test_write_speed(file_path="test_file.dat", file_size=1 * 1024 * 1024 * 1024):
    """Tests the write speed of creating a file with specified size using io_uring."""
    data = b"0" * file_size
    start_time = time.time()
    io_uring_write(file_path, data)
    end_time = time.time()

    duration = end_time - start_time
    speed = file_size / duration / (1024 * 1024)  # Convert to MB/s

    print(f"Write speed: {speed:.2f} MB/s")
    os.remove(file_path)  # Clean up after test
    print(duration)
if __name__ == "__main__":
    test_write_speed()

Write speed: 146.98 MB/s
6.9669365882873535


In [1]:
from liburing import O_CREAT, O_RDWR, AT_FDCWD, iovec, io_uring, io_uring_get_sqe, \
                     io_uring_prep_openat, io_uring_prep_write, io_uring_prep_read, \
                     io_uring_prep_close, io_uring_submit, io_uring_wait_cqe, \
                     io_uring_cqe_seen, io_uring_cqe, io_uring_queue_init, io_uring_queue_exit, \
                     io_uring_sqe_set_data64, trap_error


def open(ring, cqe, path, flags, mode=0o660, dir_fd=AT_FDCWD):
    _path = path if isinstance(path, bytes) else str(path).encode()
    # if `path` is relative and `dir_fd` is `AT_FDCWD`, then `path` is relative
    # to current working directory. Also `_path` must be in bytes

    sqe = io_uring_get_sqe(ring)  # sqe(submission queue entry)
    io_uring_prep_openat(sqe, _path, flags, mode, dir_fd)
    # set submit entry identifier as `1` which is returned back in `cqe.user_data`
    # so you can keep track of submit/completed entries.
    io_uring_sqe_set_data64(sqe, 1)
    return _submit_and_wait(ring, cqe)  # returns fd


def write(ring, cqe, fd, data, offset=0):
    iov = iovec(data)  # or iovec([bytearray(data)])
    sqe = io_uring_get_sqe(ring)
    io_uring_prep_write(sqe, fd, iov.iov_base, iov.iov_len, offset)
    io_uring_sqe_set_data64(sqe, 2)
    return _submit_and_wait(ring, cqe)  # returns length(s) of bytes written


def read(ring, cqe, fd, length, offset=0):
    iov = iovec(bytearray(length))  # or [bytearray(length)]
    sqe = io_uring_get_sqe(ring)
    io_uring_prep_read(sqe, fd, iov.iov_base, iov.iov_len, offset)
    io_uring_sqe_set_data64(sqe, 3)
    _submit_and_wait(ring, cqe)  # get actual length of file read.
    return iov.iov_base


def close(ring, cqe, fd):
    sqe = io_uring_get_sqe(ring)
    io_uring_prep_close(sqe, fd)
    io_uring_sqe_set_data64(sqe, 4)
    _submit_and_wait(ring, cqe)  # no error means success!


def _submit_and_wait(ring, cqe):
    io_uring_submit(ring)  # submit entry
    io_uring_wait_cqe(ring, cqe)  # wait for entry to finish
    result = trap_error(cqe.res)  # auto raise appropriate exception if failed
    # note `cqe.res` returns results, if ``< 0`` its an error, if ``>= 0`` its the value

    # done with current entry so clear it from completion queue.
    io_uring_cqe_seen(ring, cqe)
    return result  # type: int

In [11]:
import time
def main(data):
    ring = io_uring()
    cqe = io_uring_cqe()  # completion queue entry


    io_uring_queue_init(32, ring, 0)

    fd = open(ring, cqe, 'liburing-test-file.txt', O_CREAT | O_RDWR)
    print('fd:', fd)

    start = time.time()
    length = write(ring, cqe, fd, data)
    end = time.time()
    print(end - start)
    print('wrote:', length)


    # content = read(ring, cqe, fd, length)
    # print('read:', content)

    close(ring, cqe, fd)
    print('closed.')

    io_uring_queue_exit(ring)


data = b"0" * 1 * 1024 * 1024 * 1024
print(len(data))
main(data)
del data

1073741824
fd: 50
7.837464809417725
wrote: 1073741824
closed.
